# Recursive Language Models with DSPy

## Programmatic Long-Context Processing

---

This notebook demonstrates how to use **DSPy's RLM module** - an implementation of Recursive Language Models integrated into the DSPy framework.

### What is DSPy?

[DSPy](https://github.com/stanfordnlp/dspy) is a framework for **programming with language models**, not just prompting them. Key concepts:

- **Signatures**: Declarative specifications of input/output (e.g., `"context, query -> answer"`)
- **Modules**: Composable building blocks (Predict, ChainOfThought, ReAct, RLM)
- **Optimizers**: Automatic prompt tuning and few-shot learning

### What is RLM?

**Recursive Language Models (RLMs)** solve the context length problem by:

1. Storing the context as a variable in a code execution environment
2. Letting the LLM write Python code to explore and analyze the context
3. Enabling recursive sub-LLM calls via `llm_query()` on specific chunks
4. Iterating until the model produces a final answer via `SUBMIT()`

### DSPy RLM vs Standalone RLM

| Feature | DSPy RLM | Standalone rlm |
|---------|----------|----------------|
| Integration | Full DSPy ecosystem | Standalone library |
| Interface | Signature-based | `completion()` method |
| Custom Tools | `tools={}` parameter | Not directly supported |
| Environments | Python interpreter | local, docker, modal, prime |
| Logging | Built-in verbose | RLMLogger + Visualizer |

---

## 1. Setup Instructions

### Install DSPy

DSPy's RLM is available in recent versions of the library.

In [1]:
# Verify installation
import dspy

print(f"DSPy version: {dspy.__version__}")

# Check RLM is available
try:
    from dspy import RLM
    print("DSPy RLM imported successfully!")
except ImportError:
    print("RLM not found - you may need a newer version of DSPy")
    print("Try: uv pip install --upgrade dspy")

DSPy version: 3.1.2
DSPy RLM imported successfully!


### Configure API Keys

In [2]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")
# os.environ["ANTHROPIC_API_KEY"] = getpass.getpass("Enter your Anthropic API key: ")

# Verify at least one key is set
if os.getenv("OPENAI_API_KEY"):
    print("OpenAI API key configured")
if os.getenv("ANTHROPIC_API_KEY"):
    print("Anthropic API key configured")
    
if not os.getenv("OPENAI_API_KEY") and not os.getenv("ANTHROPIC_API_KEY"):
    print("WARNING: No API keys found! Please set OPENAI_API_KEY or ANTHROPIC_API_KEY")

OpenAI API key configured


### Configure DSPy's Language Model

DSPy uses a global configuration for the default LM:

In [3]:
import dspy

# Configure the default LM for DSPy
# Using gpt-4o-mini for cost-effective demos
lm = dspy.LM(
    model="openai/gpt-4o-mini",
    api_key=os.getenv("OPENAI_API_KEY"),
)
dspy.configure(lm=lm)

print(f"Configured DSPy with: {lm.model}")

Configured DSPy with: openai/gpt-4o-mini


In [ ]:
# Alternative: Using Anthropic
# lm = dspy.LM(
#     model="anthropic/claude-sonnet-4-20250514",
#     api_key=os.getenv("ANTHROPIC_API_KEY"),
# )
# dspy.configure(lm=lm)

---

## 2. Download Long-Context Dataset

We'll use the **Complete Works of Shakespeare** from Project Gutenberg - even larger than War and Peace!

- **Size**: ~5.5 million characters
- **Approximate Tokens**: ~1.4 million tokens
- **Contains**: 37 plays, 154 sonnets, and narrative poems

In [4]:
import requests

def download_gutenberg_text(url: str) -> str:
    """Download and clean a text file from Project Gutenberg."""
    response = requests.get(url)
    response.raise_for_status()
    text = response.text
    
    # Remove Gutenberg header
    start_markers = ["*** START OF", "***START OF"]
    for marker in start_markers:
        if marker in text:
            text = text.split(marker, 1)[1]
            text = text.split("\n", 1)[1]
            break
    
    # Remove Gutenberg footer
    end_markers = ["*** END OF", "***END OF"]
    for marker in end_markers:
        if marker in text:
            text = text.split(marker, 1)[0]
            break
    
    return text.strip()

# Download Complete Works of Shakespeare
shakespeare_url = "https://www.gutenberg.org/cache/epub/100/pg100.txt"
shakespeare = download_gutenberg_text(shakespeare_url)

print(f"Downloaded Complete Works of Shakespeare")
print(f"  Characters: {len(shakespeare):,}")
print(f"  Approximate tokens: ~{len(shakespeare) // 4:,}")
print(f"  Lines: {len(shakespeare.splitlines()):,}")
print()
print("First 500 characters:")
print("-" * 50)
print(shakespeare[:500])

Downloaded Complete Works of Shakespeare
  Characters: 5,555,356
  Approximate tokens: ~1,388,839
  Lines: 196,015

First 500 characters:
--------------------------------------------------
The Complete Works of William Shakespeare

by William Shakespeare




                    Contents

    THE SONNETS
    ALL’S WELL THAT ENDS WELL
    THE TRAGEDY OF ANTONY AND CLEOPATRA
    AS YOU LIKE IT
    THE COMEDY OF ERRORS
    THE TRAGEDY OF CORIOLANUS
    CYMBELINE
    THE TRAGEDY OF HAMLET, PRINCE OF DENMARK
    THE FIRST PART OF KING HENRY THE FOURTH
    THE SECOND PART OF KING HENRY THE FOURTH
    THE LIFE OF KING HENRY THE FIFTH
    THE FIRST PART OF HENRY THE SIX


---

## 3. Understanding DSPy Signatures

DSPy uses **signatures** to define what goes in and what comes out of a module.

### Signature Syntax

```python
"input_field1, input_field2 -> output_field"
```

### How RLM Uses Signatures

For RLM, the special field `context` becomes a variable in the REPL environment:

```python
# This signature:
"context, query -> answer"

# Makes 'context' available as a Python variable in the REPL
# The LLM can write code like:
#   len(context)
#   context.split('\n')[:10]
#   'hamlet' in context.lower()
```

In [5]:
# Example signatures for different use cases

# Basic Q&A over a document
basic_sig = "context, query -> answer"

# Multiple outputs
structured_sig = "context, query -> summary, key_quotes, conclusion"

# Domain-specific
literary_sig = "text, question -> analysis, evidence, themes"

print("Signature examples:")
print(f"  Basic: {basic_sig}")
print(f"  Structured: {structured_sig}")
print(f"  Literary: {literary_sig}")

Signature examples:
  Basic: context, query -> answer
  Structured: context, query -> summary, key_quotes, conclusion
  Literary: text, question -> analysis, evidence, themes


---

## 4. Basic DSPy RLM Usage

Let's create an RLM instance and see it in action!

### RLM Parameters

| Parameter | Description | Default |
|-----------|-------------|---------|
| `signature` | Input/output specification | Required |
| `max_iterations` | Maximum REPL interaction cycles | 20 |
| `max_llm_calls` | Budget for sub-LLM queries | 50 |
| `max_output_chars` | Output truncation limit | 100,000 |
| `verbose` | Show execution details | False |
| `tools` | Custom tool functions | {} |
| `sub_lm` | Alternate LM for sub-queries | None |

In [6]:
from dspy import RLM

# Create a basic RLM instance
rlm = RLM(
    signature="context, query -> answer",
    max_iterations=15,      # Maximum REPL cycles
    max_llm_calls=30,       # Limit on sub-LLM queries
    verbose=True,           # Show execution details
)

print("RLM created with signature: context, query -> answer")

RLM created with signature: context, query -> answer


In [7]:
# Simple test with a small context
simple_context = """
The secret codes are:
- ALPHA: 42
- BETA: 17
- GAMMA: 99
- DELTA: 256

Remember: The master code is the sum of ALPHA and GAMMA.
"""

result = rlm(
    context=simple_context,
    query="What is the master code?"
)

print("\n" + "=" * 60)
print("RESULT")
print("=" * 60)
print(f"Answer: {result.answer}")

2026/01/21 12:12:20 WARNING dspy.primitives.python_interpreter: Unable to find the Deno cache dir.
2026/01/21 12:12:25 INFO dspy.predict.rlm: RLM iteration 1/15
Reasoning: I know that the master code is defined as the sum of the values associated with the variables ALPHA and GAMMA. From the provided context, ALPHA is 42 and GAMMA is 99. To find the master code, I need to compute the sum of these two values. I will write a small piece of code to add these two numbers together and print the result.
Code:
# Extracting values for ALPHA and GAMMA and calculating the master code
ALPHA = 42
GAMMA = 99
master_code = ALPHA + GAMMA
print(master_code)
2026/01/21 12:12:32 INFO dspy.predict.rlm: RLM iteration 2/15
Reasoning: Since the previous attempt to compute the master code resulted in an error regarding the execution environment, I will instead reformat my approach to ensure that I can compute the master code without directly referencing Deno. This time, I'll extract the values of ALPHA and GA


RESULT
Answer: The master code is 141, which is the sum of ALPHA (42) and GAMMA (99).


---

## 5. Long-Context Challenge: Shakespeare Analysis

Now let's tackle the Complete Works of Shakespeare - **5.5 million characters** that no traditional LLM can process directly!

The RLM will:
1. Store the entire text as the `context` variable
2. Write Python code to search, count, and analyze
3. Use `llm_query()` for semantic understanding of specific passages
4. Return the final answer via `SUBMIT()`

In [8]:
# Query requiring analysis across multiple plays
query = """
Count how many times the word 'love' appears in the complete works.
Then identify which play mentions 'love' most frequently.
Finally, provide 2 famous quotes about love from different plays.
"""

result = rlm(
    context=shakespeare,
    query=query
)

print("\n" + "=" * 60)
print("SHAKESPEARE LOVE ANALYSIS")
print("=" * 60)
print(result.answer)

2026/01/21 12:13:55 INFO dspy.predict.rlm: RLM iteration 1/15
Reasoning: To address the query about the word 'love' in the complete works of William Shakespeare, I will perform the following steps:
1. Count the total occurrences of the word 'love' in the text of the complete works.
2. Identify which of the plays mentions 'love' the most frequently and provide the count for each play.
3. Extract two famous quotes about love from different plays.

First, I will clean the `context` to isolate individual plays and then count occurrences of 'love' in the entire text and within each play. This should provide the necessary information to answer the query adequately.
Code:
# First, let's split the complete works by play and count occurrences of the word 'love'
import re
from collections import defaultdict

# Split the complete works into plays based on known play title patterns
works = context.split("\n\n")  # Assuming double new lines separate the plays
play_dict = defaultdict(str)

# Populat


SHAKESPEARE LOVE ANALYSIS
The word "love" appears a total of 1,339 times in the complete works of Shakespeare. The play that mentions "love" most frequently is "Romeo and Juliet," with a total of 622 occurrences. Here are two famous quotes about love from different plays:

1. **From "Romeo and Juliet":** "But, soft! What light through yonder window breaks? It is the east, and Juliet is the sun."

2. **From "A Midsummer Night's Dream":** "The course of true love never did run smooth."


In [9]:
# More complex query: Cross-play character comparison
comparison_query = """
Compare the villains Iago (from Othello) and Lady Macbeth (from Macbeth):
1. Find passages where each character reveals their motivations
2. Analyze their manipulation techniques
3. Compare their ultimate fates

Use specific quotes from the text as evidence.
"""

result = rlm(
    context=shakespeare,
    query=comparison_query
)

print("\n" + "=" * 60)
print("VILLAIN COMPARISON")
print("=" * 60)
print(result.answer)

2026/01/21 12:16:54 INFO dspy.predict.rlm: RLM iteration 1/15
Reasoning: To compare the villains Iago from "Othello" and Lady Macbeth from "Macbeth," I need to first extract relevant passages from the texts that showcase each character's motivations. This requires searching through the "context" variable, which contains the complete works of Shakespeare. 

I will start by focusing on Iago and look for passages that reveal his motivations. After that, I will search for passages related to Lady Macbeth and her motivations. Once both passages are retrieved, I can analyze their manipulation techniques and fates based on additional content. 

Since the context is very large, I will extract only a sample related to Iago first, and once I have that, I will proceed to Lady Macbeth.
Code:
import re

# Extracting relevant passages about Iago from the context
iago_passages = re.findall(r"(Iago.*?)(?=\.\s|$)", context, re.DOTALL)

# Displaying the found passages for Iago
iago_passages[:3]  # Show 


VILLAIN COMPARISON
{
    "Iago": {
        "Manipulation Techniques": [
            "Iago's hypocrisy is highlighted here; he shows a duplicitous nature.",
            "Iago manipulates others by emphasizing honesty, yet embodies deception.",
            "Iago's jealousy and ambition drive him to sow discord."
        ],
        "Ultimate Fate": "Iago is ultimately discovered and punished for his treachery, getting arrested and facing severe consequences."
    },
    "Lady Macbeth": {
        "Manipulation Techniques": [
            "Lady Macbeth's guilt and mental unraveling indicate her moral conflicts.",
            "She advises Macbeth to hide his intentions, showcasing her manipulative prowess.",
            "Lady Macbeth's dismissive attitude towards femininity illustrates her ambition."
        ],
        "Ultimate Fate": "Lady Macbeth succumbs to guilt and madness, leading to her tragic death, rumored to be by suicide."
    }
}


---

## 6. Multiple Output Fields

DSPy signatures support multiple output fields, which is great for structured responses:

In [10]:
# RLM with multiple output fields
rlm_structured = RLM(
    signature="context, query -> summary, key_quotes, themes",
    max_iterations=15,
    verbose=True,
)

result = rlm_structured(
    context=shakespeare,
    query="Analyze the theme of jealousy in Othello"
)

print("\n" + "=" * 60)
print("STRUCTURED OUTPUT")
print("=" * 60)
print(f"\nSUMMARY:\n{result.summary}")
print(f"\nKEY QUOTES:\n{result.key_quotes}")
print(f"\nTHEMES:\n{result.themes}")

2026/01/21 12:18:54 INFO dspy.predict.rlm: RLM iteration 1/15
Reasoning: To analyze the theme of jealousy in Othello, I should first extract relevant passages from the text of Othello. Given the extensive size of the `context`, I will begin by locating sections of the play that focus on or reference jealousy. I can then process these excerpts to identify how jealousy is portrayed, its consequences, and its significance in the narrative. I will start by querying the text of Othello for specific mentions of jealousy.
Code:
# Let's search for all occurrences of the word "jealousy" and related terms in the text of Othello.
import re

# Extracting the text for Othello from the context
othello_text = re.search(r"(THE TRAGEDY OF OTHELLO.*?)(?=(THE TRAGEDY OF|THE COMEDY OF|THE SONNETS|$))", context, re.DOTALL)

# Check if Othello was found and present a small snippet.
othello = othello_text.group(1) if othello_text else "Othello not found."
othello_snippet = othello[:1000]  # Preview the first


STRUCTURED OUTPUT

SUMMARY:
Jealousy is the central theme in William Shakespeare's *Othello*, driving the narrative toward tragedy and significantly influencing the relationships between characters. Othello's insecurities are exploited by Iago, leading to his intense jealousy, which distorts his perception of reality and fosters mistrust, ultimately resulting in catastrophic consequences. The theme explores the destructive power of jealousy, exemplifying how it can corrupt and ruin the lives of those it touches.

KEY QUOTES:
1. "O, beware, my lord, of jealousy; It is the green-eyed monster which doth mock The meat it feeds on."
2. "Why did I marry? This honest creature doubtless Sees and knows more, much more, than he unfolds."
3. "I’ll see before I doubt; when I doubt, prove; And on the proof, there is no more but this: Away at once with love or jealousy!"

THEMES:
The analysis of jealousy in *Othello* encompasses several key themes:
- Jealousy as a driving force: It propels the acti

---

## 7. Custom Tools

DSPy RLM allows you to add **custom tools** that become available in the REPL environment. This is powerful for domain-specific analysis!

In [11]:
import re
from collections import Counter

def count_word(text: str, word: str) -> int:
    """Count occurrences of a word in text (case-insensitive)."""
    pattern = rf'\b{re.escape(word)}\b'
    return len(re.findall(pattern, text, re.IGNORECASE))

def find_play_section(text: str, play_name: str) -> str:
    """Extract a specific play from the complete works."""
    # Shakespeare's plays are typically marked with their titles
    plays = text.split('<<')
    for section in plays:
        if play_name.lower() in section[:200].lower():
            return section[:50000]  # Return first 50K chars of the play
    return f"Play '{play_name}' not found"

def get_word_frequency(text: str, top_n: int = 20) -> dict:
    """Get the most frequent words in the text."""
    words = re.findall(r'\b[a-z]{4,}\b', text.lower())
    # Filter out common words
    stopwords = {'that', 'this', 'with', 'have', 'will', 'your', 'from', 
                 'they', 'been', 'were', 'said', 'each', 'which', 'their',
                 'would', 'there', 'could', 'other', 'into', 'more', 'some'}
    words = [w for w in words if w not in stopwords]
    return dict(Counter(words).most_common(top_n))

# Create RLM with custom tools
rlm_with_tools = RLM(
    signature="context, query -> answer",
    max_iterations=15,
    tools={
        "count_word": count_word,
        "find_play_section": find_play_section,
        "get_word_frequency": get_word_frequency,
    },
    verbose=True,
)

print("RLM with custom tools created!")
print("Available tools: count_word, find_play_section, get_word_frequency")

RLM with custom tools created!
Available tools: count_word, find_play_section, get_word_frequency


In [12]:
# Use the custom tools
result = rlm_with_tools(
    context=shakespeare,
    query="""
    Using the available tools:
    1. Count how many times 'death' and 'love' appear in the text
    2. Get the top 10 most frequent words
    3. What does this word frequency tell us about Shakespeare's themes?
    """
)

print("\n" + "=" * 60)
print("ANALYSIS WITH CUSTOM TOOLS")
print("=" * 60)
print(result.answer)

2026/01/21 12:21:31 INFO dspy.predict.rlm: RLM iteration 1/15
Reasoning: I need to perform three tasks based on the provided `query` using the `context` which contains the complete works of William Shakespeare. First, I'll count the occurrences of the words 'death' and 'love' in the text. Next, I'll retrieve the top 10 most frequent words from the complete works. After obtaining this information, I can analyze what these frequencies suggest about Shakespeare's themes. I will start with the first two tasks: counting the specific words and getting the word frequencies.
Code:
# Count occurrences of 'death' and 'love'
death_count = count_word(context, 'death')
love_count = count_word(context, 'love')

# Get the top 10 most frequent words
top_words = get_word_frequency(context, top_n=10)

print(f"Count of 'death': {death_count}")
print(f"Count of 'love': {love_count}")
print(f"Top 10 most frequent words: {top_words}")
2026/01/21 12:21:39 INFO dspy.predict.rlm: RLM iteration 2/15
Reasoning: 


ANALYSIS WITH CUSTOM TOOLS
I encountered persistent execution issues in the Python REPL environment due to dependencies that prevented me from counting the occurrences of the words 'death' and 'love' in Shakespeare's works, as well as analyzing word frequencies. Despite multiple attempts and various simplifications, I could not obtain any meaningful results from the text. Unfortunately, I couldn't complete the tasks as intended.


---

## 8. Using a Different Sub-LM

You can specify a different (often cheaper/faster) model for sub-LLM calls while keeping a more powerful model for the main reasoning:

In [13]:
# Create a separate LM for sub-calls (cheaper model)
sub_lm = dspy.LM(
    model="openai/gpt-4o-mini",
    api_key=os.getenv("OPENAI_API_KEY"),
)

# Use a more powerful model for main reasoning
main_lm = dspy.LM(
    model="openai/gpt-4o",
    api_key=os.getenv("OPENAI_API_KEY"),
)

# Configure DSPy with the main model
dspy.configure(lm=main_lm)

# Create RLM with sub_lm for cost optimization
rlm_dual = RLM(
    signature="context, query -> answer",
    max_iterations=15,
    sub_lm=sub_lm,  # Use cheaper model for recursive calls
    verbose=True,
)

print("Dual-model RLM configured:")
print("  Main reasoning: gpt-4o")
print("  Sub-calls: gpt-4o-mini")

Dual-model RLM configured:
  Main reasoning: gpt-4o
  Sub-calls: gpt-4o-mini


In [14]:
# Reset to single model for remaining examples
lm = dspy.LM(
    model="openai/gpt-4o-mini",
    api_key=os.getenv("OPENAI_API_KEY"),
)
dspy.configure(lm=lm)

---

## 9. Integration with DSPy Modules

One of DSPy's strengths is composability. You can combine RLM with other modules to build sophisticated pipelines:

In [15]:
class LiteraryAnalyzer(dspy.Module):
    """A literary analysis pipeline combining RLM with ChainOfThought."""
    
    def __init__(self):
        # RLM for deep text analysis
        self.rlm = RLM(
            signature="text, question -> detailed_analysis",
            max_iterations=10,
            verbose=False,  # Quieter for pipeline use
        )
        
        # ChainOfThought for summarization
        self.summarizer = dspy.ChainOfThought(
            "analysis -> executive_summary"
        )
    
    def forward(self, text: str, question: str) -> str:
        # Step 1: Deep analysis with RLM
        analysis = self.rlm(text=text, question=question)
        
        # Step 2: Summarize the analysis
        summary = self.summarizer(analysis=analysis.detailed_analysis)
        
        return summary.executive_summary

# Create the analyzer
analyzer = LiteraryAnalyzer()
print("LiteraryAnalyzer pipeline created!")

LiteraryAnalyzer pipeline created!


In [16]:
# Use the pipeline
# Using a subset for faster demo
hamlet_section = shakespeare[:200000]  # First 200K chars likely contains Hamlet

summary = analyzer(
    text=hamlet_section,
    question="What are the key themes in Hamlet and how are they developed?"
)

print("\n" + "=" * 60)
print("LITERARY ANALYSIS PIPELINE RESULT")
print("=" * 60)
print(summary)

2026/01/21 12:24:41 WARNING dspy.predict.rlm: RLM reached max iterations, using extract to get final output



LITERARY ANALYSIS PIPELINE RESULT
In Shakespeare's "Hamlet," five key themes underpin the narrative: revenge, madness, mortality, corruption, and truth versus deception. The quest for vengeance propels Hamlet into a moral quandary, while his feigned madness reveals the fragility of sanity, impacting those around him tragically. The exploration of mortality prompts deep existential questions, particularly in Hamlet's introspective soliloquies. The theme of corruption illustrates the decline of Denmark’s moral integrity through Claudius's actions, reflecting the personal decay experienced by characters. Lastly, the struggle for truth amid pervasive deceit illustrates a world where clarity is elusive. These interconnected themes collaboratively illuminate the complexities of the human experience, ultimately guiding the play's tragic trajectory.


---

## 10. Comparison: Direct LLM vs RLM

Let's demonstrate why RLM is necessary for long-context tasks:

In [17]:
import time

# Test query
test_query = "How many plays are in this collection? List them."

# Test 1: Direct LM call with truncated context
print("Test 1: Direct LM (must truncate to ~50K chars)")
print("-" * 50)
start = time.time()

try:
    # Traditional approach: truncate to fit context window
    truncated = shakespeare[:50000]
    direct = dspy.Predict("context, query -> answer")
    direct_result = direct(context=truncated, query=test_query)
    print(f"Result: {direct_result.answer[:300]}...")
    print(f"NOTE: Only analyzed {len(truncated):,} of {len(shakespeare):,} characters!")
except Exception as e:
    print(f"Failed: {e}")

print(f"Time: {time.time() - start:.2f}s\n")

# Test 2: RLM with full context
print("Test 2: RLM (full 5.5M character text)")
print("-" * 50)
start = time.time()

rlm_result = rlm(
    context=shakespeare,
    query=test_query
)

print(f"\nResult: {rlm_result.answer[:500]}..." if len(rlm_result.answer) > 500 else f"\nResult: {rlm_result.answer}")
print(f"\nTime: {time.time() - start:.2f}s")
print(f"Analyzed full {len(shakespeare):,} characters!")

Test 1: Direct LM (must truncate to ~50K chars)
--------------------------------------------------
Result: The collection contains a total of 37 plays. Here is the list:

1. THE SONNETS
2. ALL’S WELL THAT ENDS WELL
3. THE TRAGEDY OF ANTONY AND CLEOPATRA
4. AS YOU LIKE IT
5. THE COMEDY OF ERRORS
6. THE TRAGEDY OF CORIOLANUS
7. CYMBELINE
8. THE TRAGEDY OF HAMLET, PRINCE OF DENMARK
9. THE FIRST PART OF KING...
NOTE: Only analyzed 50,000 of 5,555,356 characters!
Time: 16.27s

Test 2: RLM (full 5.5M character text)
--------------------------------------------------


2026/01/21 12:25:26 INFO dspy.predict.rlm: RLM iteration 1/15
Reasoning: I have a string that contains the complete works of William Shakespeare, including the titles of his plays. The query asks for the number of plays in this collection and requests a list of those plays. To respond accurately, I need to extract the titles of the plays from the provided `context`. Since the works contain various components, I will look for a specific pattern in the text that identifies the plays' titles, which typically appear after specific line breaks or within certain formatting styles. After identifying the titles, I will count them and list them.
Code:
# Extracting the plays from the context
import re

# Splitting the context to identify sections that may contain play titles 
# The titles generally follow after the keyword "Contents" and are newline separated
contents_section = context.split("Contents")[1]

# Using regex to find titles - assuming they are positioned at the start of lines
# Title


Result: The complete works of William Shakespeare include 37 plays. Here is the list of those plays:

1. The Comedy of Errors
2. The Taming of the Shrew
3. Henry VI, Part 1
4. Henry VI, Part 2
5. Henry VI, Part 3
6. Richard III
7. The Two Gentlemen of Verona
8. Love's Labour's Lost
9. Romeo and Juliet
10. A Midsummer Night's Dream
11. The Merchant of Venice
12. Much Ado About Nothing
13. As You Like It
14. Twelfth Night
15. The Winter's Tale
16. King John
17. Richard II
18. Henry IV, Part 1
19. Henry IV...

Time: 116.36s
Analyzed full 5,555,356 characters!


---

## 11. Available REPL Functions

When the LLM writes code in DSPy RLM, it has access to:

### Standard Functions
- `print()` - Output to stdout (visible in verbose mode)
- Standard Python builtins (len, str, int, etc.)

### RLM Special Functions
- `llm_query(prompt: str) -> str` - Make a sub-LLM call
- `llm_query_batched(prompts: List[str]) -> List[str]` - Concurrent sub-calls
- `SUBMIT(answer)` - Return the final answer

### Your Custom Tools
- Any functions passed via the `tools={}` parameter

In [18]:
# Demonstrate sub-LLM calls
demo_rlm = RLM(
    signature="context, query -> answer",
    max_iterations=10,
    verbose=True,
)

result = demo_rlm(
    context="""
    Chapter 1: The boy who lived.
    Mr. and Mrs. Dursley, of number four, Privet Drive, were proud to say 
    that they were perfectly normal, thank you very much.
    
    Chapter 2: The Vanishing Glass.
    Nearly ten years had passed since the Dursleys had woken up to find 
    their nephew on the front step.
    """,
    query="Use llm_query to get a one-sentence summary of each chapter, then combine them."
)

print("\n" + "=" * 60)
print("RESULT")
print("=" * 60)
print(result.answer)

2026/01/21 12:27:23 INFO dspy.predict.rlm: RLM iteration 1/10
Reasoning: I have the context containing excerpts from a text that includes chapters and the main characters. The query instructs me to use the `llm_query` function to generate a one-sentence summary for each chapter and then to combine these summaries into a single output. To start, I will first split the context into chapters to process each individually and then summarize each chapter with `llm_query`.
Code:
# Splitting the context into chapters based on the delimiter 'Chapter'
chapters = context.split("Chapter")[1:]  # Skipping the first split element which is empty
chapter_summaries = []

# Generate a summary for each chapter
for chapter in chapters:
    # Clean up chapter text and format for summarization
    chapter_text = "Chapter" + chapter.strip()  # Keep chapter number for reference
    summary = llm_query(chapter_text)  # Querying summarization
    chapter_summaries.append(summary)

# Printing the summaries for i


RESULT
The first chapter introduces the Dursley family and hints at the unusual circumstances surrounding a boy who is not normal. The second chapter recalls a significant moment in the boy’s childhood, involving a mysterious vanishing glass.


---

## 12. Comparison Summary: DSPy RLM vs Standalone rlm

| Aspect | DSPy RLM | Standalone rlm |
|--------|----------|----------------|
| **Installation** | `uv pip install dspy` | `uv pip install -e ./rlm` |
| **Interface** | `rlm(context=..., query=...)` | `rlm.completion(context, root_prompt=query)` |
| **Output** | `result.answer` (or custom fields) | `result.response` |
| **Final Answer** | `SUBMIT(answer)` | `FINAL(answer)` or `FINAL_VAR(var)` |
| **Custom Tools** | `tools={"name": func}` | Not directly supported |
| **Environments** | Python interpreter | local, docker, modal, prime |
| **Logging** | `verbose=True` | `RLMLogger` + `verbose=True` |
| **Visualization** | None built-in | Dedicated visualizer app |
| **DSPy Integration** | Full (signatures, optimizers) | Standalone |

### When to Use Each

**Use DSPy RLM when:**
- You're already using DSPy for other modules
- You need custom tools in the REPL
- You want to use DSPy optimizers
- You prefer signature-based interfaces

**Use Standalone rlm when:**
- You need isolated execution (Docker, Modal, Prime)
- You want detailed trajectory visualization
- You need fine-grained control over the REPL environment
- You're not using DSPy elsewhere

---

## 13. Conclusion

### What We Learned

1. **DSPy RLM** integrates seamlessly with the DSPy framework
2. **Signatures** define structured inputs and outputs
3. **Custom tools** extend RLM capabilities for domain-specific tasks
4. **Composability** lets you build sophisticated pipelines
5. **Sub-LM optimization** enables cost-effective processing

### Key Takeaways

- RLM solves the context length problem without sacrificing understanding
- The programmatic approach enables systematic analysis of massive texts
- DSPy's module system makes it easy to integrate RLM into larger systems

### Resources

- [DSPy GitHub](https://github.com/stanfordnlp/dspy)
- [DSPy Documentation](https://dspy-docs.vercel.app/)
- [RLM Paper](https://arxiv.org/abs/2512.24601)
- [Standalone rlm Repository](https://github.com/alexzhang13/rlm)